In [15]:
import cv2

In [16]:
#our dataset: http://pascal.inrialpes.fr/data/human/

In [17]:
#load in image and detect face(s), https://realpython.com/face-recognition-with-python/ (need adjust size of squares to include hair)

# Create the haar cascade
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

TEST_PHOTO = "../OtherFinalProjectStuff/INRIAPerson/Train/pos/person_139.png"

# Read the image
image = cv2.imread(TEST_PHOTO)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=3,
    minSize=(15, 15),
    flags = cv2.CASCADE_SCALE_IMAGE #https://stackoverflow.com/questions/41341409/where-is-cv-haar-scale-image-in-opencv-3-1-0-with-python-3-5
)

print("Found {0} faces!".format(len(faces)))

# Draw a rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

cv2.imshow("Faces found", image)
cv2.waitKey(0)

Found 3 faces!


-1

In [18]:
for (x,y,w,h) in faces:
    print(x, y, w, h)

87 133 24 24
300 130 32 32
211 160 30 30


In [19]:
#faces[0][0] = 442
#faces[0][1] = 353
#faces[0][2] = 50
#faces[0][3] = 50

#import numpy as np
#faces = np.delete(faces, 0, axis=0)
#print(faces)

#faces[0][2] = 32
#faces[0][3] = 32
#faces[0][0] = 226
#faces[0][1] = 94

#faces = np.delete(faces, 0, axis=0)

#print(faces)

In [20]:
#image = cv2.imread(TEST_PHOTO)

#Need hardcoded offsets for this! For the kid, its +10, -5

#for (x, y, w, h) in faces:
#    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

#cv2.imshow("Faces found", image)
#cv2.waitKey(0)

In [21]:
image = cv2.imread(TEST_PHOTO)

index = 1
for (x, y, w, h) in faces:
    cv2.imwrite('roi' + str(index) + '.png', image[y:y+h, x:x+w]) #https://stackoverflow.com/questions/9084609/how-to-copy-a-image-region-using-opencv-in-python
    index += 1

In [22]:
import numpy as np
from tensorflow.keras.layers import Add, Conv2D, Input, Lambda
from tensorflow.keras.models import Model

DIV2K_RGB_MEAN = np.array([0.4488, 0.4371, 0.4040]) * 255

def edsr(scale, num_filters=64, num_res_blocks=8, res_block_scaling=None):
    """Creates an EDSR model."""
    x_in = Input(shape=(None, None, 3))
    x = Lambda(normalize)(x_in)

    x = b = Conv2D(num_filters, 3, padding='same')(x)
    for i in range(num_res_blocks):
        b = res_block(b, num_filters, res_block_scaling)
    b = Conv2D(num_filters, 3, padding='same')(b)
    x = Add()([x, b])

    x = upsample(x, scale, num_filters)
    x = Conv2D(3, 3, padding='same')(x)

    x = Lambda(denormalize)(x)
    return Model(x_in, x, name="edsr")


def res_block(x_in, filters, scaling):
    """Creates an EDSR residual block."""
    x = Conv2D(filters, 3, padding='same', activation='relu')(x_in)
    x = Conv2D(filters, 3, padding='same')(x)
    if scaling:
        x = Lambda(lambda t: t * scaling)(x)
    x = Add()([x_in, x])
    return x


def upsample(x, scale, num_filters):
    def upsample_1(x, factor, **kwargs):
        """Sub-pixel convolution."""
        x = Conv2D(num_filters * (factor ** 2), 3, padding='same', **kwargs)(x)
        return Lambda(pixel_shuffle(scale=factor))(x)

    if scale == 2:
        x = upsample_1(x, 2, name='conv2d_1_scale_2')
    elif scale == 3:
        x = upsample_1(x, 3, name='conv2d_1_scale_3')
    elif scale == 4:
        x = upsample_1(x, 2, name='conv2d_1_scale_2')
        x = upsample_1(x, 2, name='conv2d_2_scale_2')

    return x

def pixel_shuffle(scale):
    return lambda x: tf.nn.depth_to_space(x, scale)


def normalize(x):
    return (x - DIV2K_RGB_MEAN) / 127.5


def denormalize(x):
    return x * 127.5 + DIV2K_RGB_MEAN

In [23]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
#send face(s) through super resolution network, (????? TODO, train on Colab, load here, find LR/SR face dataset as well)

# EDSR baseline as described in the EDSR paper (1.52M parameters)
model_edsr = edsr(scale=4, num_res_blocks=16)

# Adam optimizer with a scheduler that halfs learning rate after 200,000 steps
optim_edsr = Adam(learning_rate=PiecewiseConstantDecay(boundaries=[200000], values=[1e-4, 5e-5]))

# Compile and train model for 300,000 steps with L1 pixel loss
model_edsr.compile(optimizer=optim_edsr, loss='mean_absolute_error')
 
model_edsr.load_weights('weights-edsr-40E.h5')

import imageio

index = 1
y1 = []
for (x, y, w, h) in faces:
    im = imageio.imread('roi' + str(index) + '.png')
    y1.append(model_edsr.predict(tf.reshape(tf.convert_to_tensor(im), [1,w,h,3]),steps=1))
    index += 1

In [24]:
from PIL import Image
import numpy as np

index = 0
for (x,y,w,h) in faces:
    PIL_image = Image.fromarray(np.asarray(y1[index]).reshape(4*w,4*h,3).astype('uint8'), 'RGB')
    PIL_image.save('test' + str(index+1) + '.png')
    index += 1

In [25]:
#overlay big face on OG image: https://stackoverflow.com/questions/40895785/using-opencv-to-overlay-transparent-image-onto-another-image?fbclid=IwAR2FxAinlVh8GOKCZmQW0kZMOmAEt_caonwHTHPI3cDAiT3Yo-tJGTOmufs
imageOG = cv2.imread(TEST_PHOTO)
cv2.imwrite("combined.png", imageOG)

imageOG = cv2.imread("combined.png")

index = 1

for (x, y, w, h) in faces:
    bigHead = cv2.imread("test" + str(index) + ".png")
    rows,cols,channels = bigHead.shape
    imageOG[y-int((3/4)*cols):y-int((3/4)*cols)+cols, x-int((3/8)*rows):x-int((3/8)*rows)+rows] = bigHead #overlay
    cv2.imwrite('combined.png', imageOG)
    index += 1

In [26]:
#cartoonify image and display
#https://analyticsindiamag.com/converting-image-into-a-pencil-sketch-in-python/
#OR
#https://analyticsindiamag.com/converting-an-image-to-a-cartoon/?fbclid=IwAR1KsgK4sj3mpkUU-kIolQS1kUFnkkCo5bLDeq4BFPvfOzXRa-_gHF_PbWw

combined = cv2.imread("combined.png")
img_gray = cv2.cvtColor(combined, cv2.COLOR_BGR2GRAY)
img_invert = cv2.bitwise_not(img_gray)
img_smoothing = cv2.GaussianBlur(img_invert, (31, 31),sigmaX=0, sigmaY=0)

def dodgeV2(x, y):
    return cv2.divide(x, 255 - y, scale=256)

final_img = dodgeV2(img_gray, img_smoothing)

cv2.imwrite('final.png', final_img)

True

In [31]:
#https://stackoverflow.com/questions/42594993/gradient-mask-blending-in-opencv-python
def alphaBlend(img1, img2, mask):
    """ alphaBlend img1 and img 2 (of CV_8UC3) with mask (CV_8UC1 or CV_8UC3)
    """
    if mask.ndim==3 and mask.shape[-1] == 3:
        alpha = mask/255.0
    else:
        alpha = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)/255.0
    blended = cv2.convertScaleAbs(img1*(1-alpha) + img2*alpha)
    return blended

final = cv2.imread("final.png")
cv2.imwrite('finalmasked.png', final)

index = 1
for (x, y, w, h) in faces:
    final = cv2.imread("finalmasked.png")
    bigHead = cv2.imread("test" + str(index) + ".png")
    rows,cols,channels = bigHead.shape
    H,W = final.shape[:2]
    mask = np.zeros((H,W), np.uint8)
    cv2.rectangle(mask, (x-int((3/8)*rows),y-int((3/4)*cols)), (x-int((3/8)*rows)+rows,y-int((3/4)*cols)+cols), (255,255,255), 8)
    mask = cv2.GaussianBlur(mask, (21, 21),sigmaX=0, sigmaY=0)

    blured = cv2.GaussianBlur(final, (31,31),sigmaX=0, sigmaY=0)

    blended1 = alphaBlend(final, blured, mask)

    cv2.imwrite('finalmasked.png', blended1)
    index += 1
    
    #cv2.imshow("blened1", blended1);
    #cv2.imshow("mask", mask);
    #cv2.waitKey();cv2.destroyAllWindows()

In [28]:
#OTHER ideas include changing what we big-ify, changing the background (to a touristy place) and the weather...